# CSS844 Ear Size/Shape Group
**Members:** Joshua Kaste, Rie Sadohara, Hannah Jeffreys

The code below takes a list of images of filenames for pictures of corn and performs the following tasks: 
1. Identifies the width and length of the tray the corn is on and uses this to:
    **(1)** Calculate a pixel-to-cm conversion based off user-provided dimensions for the tray, and 
    **(2)** Crop the images to just a bit larger than the size of the tray.
2. Crops the individual ears of corn from the images generated in Step 1.
3. Estimates the dimensions of the cropped ears of corn and then reports these out to an Excel spreadsheet.

# Values to modify:

The cell below contains variables you can modify to get the output you are looking for. 

In [ ]:
fileNames = ['IMG_0141.jpg','IMG_0505.jpg','IMG_0992.jpg','IMG_0997.jpg','IMG_8242.jpg',
'IMG_8243.jpg','IMG_8245.jpg','IMG_8247.jpg','IMG_8249.jpg','IMG_8251.jpg',
'IMG_8253.jpg','IMG_8255.jpg','IMG_8256.jpg','IMG_8258.jpg','IMG_8260.jpg',
'IMG_8271.jpg','IMG_8272.jpg','IMG_8273.jpg','IMG_8274.jpg','IMG_8276.jpg',
'IMG_8277.jpg','IMG_8278.jpg','IMG_8279.jpg','IMG_8280.jpg','IMG_8281.jpg',
'IMG_8282.jpg','IMG_8283.jpg','IMG_8284.jpg','IMG_8286.jpg','IMG_8288.jpg',
'IMG_8292.jpg','IMG_8293.jpg','IMG_8305.jpg','IMG_8311.jpg','IMG_8312.jpg',
'IMG_8313.jpg','IMG_8314.jpg','IMG_8316.jpg','IMG_8317.jpg','IMG_8319.jpg',
'IMG_8321.jpg','IMG_8322.jpg','IMG_8323.jpg','IMG_8324.jpg','IMG_8325.jpg',
'IMG_8327.jpg','IMG_8328.jpg','IMG_8329.jpg','IMG_8331.jpg','IMG_8332.jpg',
'IMG_8333.jpg','IMG_8334.jpg','IMG_8335.jpg','IMG_8336.jpg','IMG_8338.jpg',
'IMG_8339.jpg','IMG_8340.jpg','IMG_8341.jpg','IMG_8342.jpg','IMG_8343.jpg',
'IMG_8344.jpg','IMG_8345.jpg','IMG_8346.jpg','IMG_8347.jpg','IMG_8348.jpg',
'IMG_8349.jpg','IMG_8350.jpg','IMG_8356.jpg'] # e.g. fileNames = ['IMG_8277.jpg']

top_bottom_in_cm = 38.5 # length of tray
left_right_in_cm = 29.4 # width of tray
cropped_images_prefixes = 'PrePublish_testing_crops_'
segmented_corn_prefixes = 'plot_of_'
num_corn = 5 # change this if the set of images you are processing has a different number of ears
resultsFileName = 'Results.xlsx' 


# Initial Crop to Tray (modified from Miles Roberts' code)

In [ ]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
import skimage
from skimage.morphology import remove_small_holes
from skimage.morphology import remove_small_objects
from skimage import exposure #histogram equalization
import colorsys #To convert to rbg to hsv color space
import matplotlib.colors as colors
import os #For getting list of files
from scipy import ndimage #For performing erosion and dilation

final_fileNames = []
#Define thresholds for isolating tray in photos
hmin = -0.01
hmax = 1.01
smin = -0.01
smax = 0.08
vmin = -0.01
vmax = 1.01

#Function for calculating run lengths in a binary array
#function is from: https://stackoverflow.com/questions/1066758/find-length-of-sequences-of-identical-values-in-a-numpy-array-run-length-encodi
def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

conversion_dict = {}        

#Loop over photos and crop them
for fileName in fileNames:
    #Load picture
    im = imread(fileName)
    
    #Histogram equalization
    im2 = exposure.equalize_hist(im)
    
    #convert from rgb to hsv color space, pull out matrices
    hsv = colors.rgb_to_hsv(im2)
    h = hsv[:,:,0]; #hue matrix
    s = hsv[:,:,1]; #saturation matrix
    v = hsv[:,:,2]; #value matrix (i.e. brightness)
    
    #Convert to binary image based on thresholds
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    b_img = (b_img & 
        (s > smin) & (s < smax) & 
        (v > vmin) & (v < vmax));
    
    #Clean up binary image with erosion and dilation
    b4 = skimage.morphology.remove_small_holes(b_img,area_threshold=50000) # removing holes
    b5 = skimage.morphology.remove_small_holes(b4,area_threshold=10000) # removing holes again
    b6 = skimage.morphology.remove_small_holes(b5,area_threshold=10000) # removing even more holes
    b8 = remove_small_objects(b6,min_size=50000) #removing any chaff 
    
    #Label objects in binary image
    lab, num_features = ndimage.measurements.label(b8)
    
    #Sum togther rows and columns of binary array to determine which pixels represent the tray (labeled as object 1) 
    a1 = np.sum(lab==1,axis=1)
    a0 = np.sum(lab==1,axis=0)
    
    #Convert binary arrays to logical arrays. Now just need to find longest run of False elements in each array
    al0 = a0 > min(a0)
    al1 = a1 > min(a1)
    
    #Calculate run lengths
    runLengths0 = rle(al0)
    runLengths1 = rle(al1)
    
    #Focus on columns
    ##Find index of where longest run begins
    runs0 = runLengths0[0]
    positions0 =  runLengths0[1]
    maxRun0 = max(runs0)
    result = np.where(runs0 == maxRun0)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startCol = positions0[index].tolist()[0][0] - 120
    if startCol < 0: # we don't want our indices to fall outside the edges of the image, so we check
        startCol = 0
    endCol = startCol + maxRun0 + 240
    if endCol > im2.shape[1]:
        endCol = im2.shape[1]
    
    #Focus on rows
    ##Find index of where longest run begins
    runs1 = runLengths1[0]
    positions1 =  runLengths1[1]
    maxRun1 = max(runs1)
    result = np.where(runs1 == maxRun1)
    ##Calculate where longest run ends
    index = np.asarray(result)
    startRow = positions1[index].tolist()[0][0] - 120
    if startRow < 0:
        startRow = 0
    endRow = startRow + maxRun1 + 240
    if endRow > im2.shape[0]:
        endRow = im2.shape[0]
    # calculate conversion factor by relating run lengths identified above (maxRun0 is columns and maxRun1 is rows)
    # and updating a dictionary, called conversion_dict, with a fileName to conversion factor key:value relationship.
    # Note that we don't want to overwrite our files, so we create a 'finalFilename' ahead of time, use that to
    # create the dictionary, and then use it to also export our new images. Additionally, we take the average of the
    # pixel-to-cm relationships for extra robustness
    
    finalFilename = cropped_images_prefixes + fileName

    conversion_top_bottom = top_bottom_in_cm / maxRun0
    conversion_left_right = left_right_in_cm / maxRun1
    average_conversion = (conversion_top_bottom + conversion_left_right) / 2
    new_addition = {finalFilename:average_conversion}
    conversion_dict.update(new_addition)
    final_fileNames.append(finalFilename)
    #Crop and save image
    im3 = im2[startRow:endRow,startCol:endCol]
    imsave(finalFilename, im3)

# Defining our segmentation function

In [ ]:
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
import skimage
import skimage.morphology
from scipy.ndimage.measurements import label
from scipy.ndimage.measurements import find_objects
from skimage.morphology import remove_small_objects
from PIL import Image
from matplotlib import cm

def segment_and_crop(img,num_corn):
    
    for x in range(30):
    
    # segmentation by HSV, as we've done before
    
        hmin = 0.05
        hmax = 0.20
        smin = 0.3
        smax = 1.01
        vmin = 0.6
        vmax = 1.01


        hsv = colors.rgb_to_hsv(img)
        h = hsv[:,:,0];
        s = hsv[:,:,1];
        v = hsv[:,:,2];

        # trick because the color space wraps
        if hmin > hmax:
            b_img = (h > hmin) | (h < hmax)
        else:
            b_img = (h > hmin) & (h < hmax);


        b_img = (b_img & 
            (s > smin) & (s < smax) & 
            (v > vmin) & (v < vmax));


        plt.imshow(b_img) # just showing the image
        b_img = ndimage.binary_erosion(b_img,iterations = x) # eroding i times; we increase the erosion each time because i 
        # only becomes 1+ (i.e. we only erode) if we fail a check at the end that's looking for five independent ears
        b4 = skimage.morphology.remove_small_holes(b_img,area_threshold=10000) # removing holes
        b5 = skimage.morphology.remove_small_holes(b4,area_threshold=10000) # removing holes again
        b6 = skimage.morphology.remove_small_holes(b5,area_threshold=10000) # removing even more holes
        #b7 = ndimage.binary_erosion(b6, iterations = 1)
        b8 = remove_small_objects(b6,min_size=50000) #removing any chaff 

        labeled_array, num_features = label(b8) # labeling the resulting cleaned up binary image

        objects = []

        for i in range(1000):
            try:
                objects.append(find_objects(labeled_array)[i]) # adding these labeled things to a list
            except:
                break

        slices = []        

        for i in range(len(objects)):
            slices.append(labeled_array[objects[i]]) # extracting slices from these objects

        sizes = np.array([])

        for i in range(len(slices)):
            sizes = np.append(sizes,slices[i].size) # getting the sizes of those slices

        while i <num_corn+1: # we need this loop because in early iterations, everything might be a blob, so it'll throw an error if we don't have five discrete ears
            try:
                ind = np.argpartition(sizes,-i)[-i:] 
            except:
                i = i -1
                ind = np.argpartition(sizes,-i)[-i:]
                break
            i = i + 1

        top_five_slices = [] 
        top_five_slices_ordered = []
        
        for i in range(len(ind)):
            top_five_slices.append(slices[ind[i]]) #adding our largest slices to a list

        for i in range(len(slices)):
            if slices[i] in top_five_slices:
                top_five_slices_ordered.append(slices[i])
        
        for i in range(len(top_five_slices_ordered)):
            plt.imshow(top_five_slices_ordered[i]) # plotting each one

        if len(top_five_slices_ordered)>=num_corn: # if we have five discrete objects, we proceed. Otherwise, we continue the loop until we have what we came for
            for y in range(len(top_five_slices_ordered)): # we've eroded the image a defined amount earlier, so we compensate by dilating and then removing small holes again
                to_dilate = top_five_slices_ordered[y]
                top, bottom, left, right = [150]*4
                import cv2
                extended_image = cv2.copyMakeBorder(to_dilate, top, bottom, left, right, cv2.BORDER_CONSTANT, value=False)
                extended_dilated_image = ndimage.binary_dilation(extended_image,iterations = x)
                extended_dilated_image = remove_small_objects(extended_dilated_image,min_size=50000) #removing any chaff 
                extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000) # removing holes
                extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000) # removing holes again
                extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000)
                
                # redoing the segmentation
                
                labeled_array, num_features = label(extended_dilated_image) # labeling the resulting cleaned up binary image

                objects = []

                for i in range(1000):
                    try:
                        objects.append(find_objects(labeled_array)[i]) # adding these labeled things to a list
                    except:
                        break

                slices = []        

                for i in range(len(objects)):
                    slices.append(labeled_array[objects[i]]) # extracting slices from these objects
                
                top_five_slices_ordered[y] = slices[0]
            break
        
        elif x == 30: #if we've reached the last iteration and the loop and haven't been successful yet, we just add everything
            # whose dimensions more or less look like corn and then report out only those
            
            hsv = colors.rgb_to_hsv(img)
            h = hsv[:,:,0];
            s = hsv[:,:,1];
            v = hsv[:,:,2];

            # trick because the color space wraps
            if hmin > hmax:
                b_img = (h > hmin) | (h < hmax)
            else:
                b_img = (h > hmin) & (h < hmax);


                b_img = (b_img & 
                    (s > smin) & (s < smax) & 
                    (v > vmin) & (v < vmax));

                # only becomes 1+ (i.e. we only erode) if we fail a check at the end that's looking for five independent ears
                b4 = skimage.morphology.remove_small_holes(b_img,area_threshold=10000) # removing holes
                b5 = skimage.morphology.remove_small_holes(b4,area_threshold=10000) # removing holes again
                b6 = skimage.morphology.remove_small_holes(b5,area_threshold=10000) # removing even more holes
                #b7 = ndimage.binary_erosion(b6, iterations = 1)
                b8 = remove_small_objects(b6,min_size=100000) #removing any chaff 

                labeled_array, num_features = label(b8) # labeling the resulting cleaned up binary image
                
                objects = []

                for i in range(1000):
                    try:
                        objects.append(find_objects(labeled_array)[i]) # adding these labeled things to a list
                    except:
                        break

                slices = []        

                for i in range(len(objects)):
                    slices.append(labeled_array[objects[i]]) # extracting slices from these objects

                sizes = np.array([])

                for i in range(len(slices)):
                    sizes = np.append(sizes,slices[i].size) # getting the sizes of those slices

                while i <num_corn+1: 
                    try:
                        ind = np.argpartition(sizes,-i)[-i:] 
                    except:
                        i = i -1
                        ind = np.argpartition(sizes,-i)[-i:]
                        break
                    i = i + 1

                top_five_slices = [] 
                top_five_slices_ordered = []

                for i in range(len(slices)):
                    top_five_slices.append(slices[ind[i]]) #adding our largest slices to a list

                for i in range(len(slices)):
                    if slices[i] in top_five_slices:
                        top_five_slices_ordered.append(slices[i])
                
                for y in range(len(top_five_slices_ordered)):
                    to_dilate = top_five_slices_ordered[y]
                    top, bottom, left, right = [150]*4
                    import cv2
                    extended_image = cv2.copyMakeBorder(to_dilate, top, bottom, left, right, cv2.BORDER_CONSTANT, value=False)
                    extended_dilated_image = ndimage.binary_dilation(extended_image,iterations = x)
                    extended_dilated_image = remove_small_objects(extended_dilated_image,min_size=50000) #removing any chaff 
                    extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000) # removing holes
                    extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000) # removing holes again
                    extended_dilated_image = skimage.morphology.remove_small_holes(extended_dilated_image,area_threshold=10000)

                    # redoing the segmentation

                    labeled_array, num_features = label(extended_dilated_image) # labeling the resulting cleaned up binary image

                    objects = []

                    for i in range(1000):
                        try:
                            objects.append(find_objects(labeled_array)[i]) # adding these labeled things to a list
                        except:
                            break

                    slices = []        

                    for i in range(len(objects)):
                        slices.append(labeled_array[objects[i]]) # extracting slices from these objects

                    top_five_slices_ordered[y] = slices[0]
            break
            
        else: 
            continue
        
            
    return top_five_slices_ordered

# Running our segment function on every image we cropped earlier, followed by length/width estimation and the export of the results file

In [ ]:
import pandas as pd

ears = []

for i in range(num_corn):
    ear_to_add = 'E{0}'.format(i+1)
    ears.append(ear_to_add)

ears_to_report = []
names = []
first_dimensions_to_report = []
second_dimensions_to_report = []
flag = []


for i in range(len(final_fileNames)):
    image = imread(final_fileNames[i])
    image = exposure.equalize_hist(image) # equalizing the exposure
            
    top_five = segment_and_crop(image,num_corn)
    try:
        
        fig, ax = plt.subplots(len(top_five),1)
        
        for y in range(len(top_five)):
            ax[y].imshow(top_five[y])
    
        plot_name = segmented_corn_prefixes + final_fileNames[i]

        plt.savefig(plot_name)
        plt.close()

        for n in range(len(top_five)):
            names.append(final_fileNames[i])
        
        for x in range(len(top_five)):
            ears_to_report.append(ears[x])
        
        dimensions = []
        
        for j in range(len(top_five)):
            first_dimension = top_five[j].shape[0] * conversion_dict[final_fileNames[i]]
            second_dimension = top_five[j].shape[1] * conversion_dict[final_fileNames[i]]
            first_dimensions_to_report.append(first_dimension)
            second_dimensions_to_report.append(second_dimension)
            if first_dimension >= 8 or second_dimension >= 30 or first_dimension <= 4 or second_dimension <=10:
                flag.append('Flagged')
            else:
                flag.append('OK')
    except:
        continue
d = {'Names':names,'Ear':ears_to_report,'First dimension':first_dimensions_to_report,'Second dimension':second_dimensions_to_report,'Flag':flag}
to_export = pd.DataFrame(d)
to_export.to_excel(resultsFileName)